## Making the trianable ansatz

In [3]:
import numpy as np

In [8]:
import numpy as np
from scipy.optimize import minimize
import pennylane as qml

I = np.eye(2)
n_qubits = 6
size_of_vec = 2**n_qubits
num_layers = 5

from utils import DFS
inputStates, expectedStates = DFS().getInitialTargetStates()

# Utilizing a minimization function suggested by ChatPGT and modified for our specific problem

In [9]:
import numpy as np
from scipy.optimize import minimize
import pennylane as qml


Udot = lambda s1, U, s2 : np.dot(np.conjugate(np.transpose(s1)),np.matmul(U,s2))

def nestedKron(*args): # use "*args" to access an array of inputs
    assert len(args) >= 2
    temp = args[0]
    for arg in args[1:]:
        temp = np.kron(temp, arg)
    return temp

def get_random_weights(num_layers):
    return 2 * np.pi * np.random.random(size=(num_layers, 5)) - np.pi

def U_ex(p):
    from scipy.linalg import expm
    X = [[0,1],[1,0]]
    Y = np.array([[0,-1j],[1j,0]], dtype=np.complex128)
    Z = [[1,0],[0,-1]]

    H_ex = (1/4)*(np.kron(X,X) + np.kron(Y,Y) + np.kron(Z,Z))
    # print(f'H_ex.type = {type(H_ex)}')
    U_exchange = expm(-1j*p*H_ex) # p is now -pi to pi
    return np.array(U_exchange)

def single_layer_U(layer_weights):
    """Trainable circuit block."""
    firstPart = nestedKron(U_ex(layer_weights[0]), U_ex(layer_weights[1]), U_ex(layer_weights[2]))
    secondPart = nestedKron(I, U_ex(layer_weights[3]), U_ex(layer_weights[4]), I)
    return np.matmul(secondPart, firstPart)

def get_matrix(weights):
    totalMatrix = np.eye(size_of_vec)
    for layer_weights in weights:
        mat = single_layer_U(layer_weights)
        totalMatrix = np.matmul(totalMatrix, mat)
    return totalMatrix

def square_loss(y_true, y_pred):
    loss = 0
    for i in range(len(expectedStates)):
        # c = np.dot(np.conjugate(expectedStates[i]), predictedStates[i])
        # c_2 = self.amp_sqrd(c)
        fidelity = qml.math.fidelity_statevector(y_true[i], y_pred[i])
        loss += (1 - fidelity) ** 2
    loss /= len(expectedStates)
    return 0.5*loss

def f_cnot_loss(y_true, y_pred):
    loss = 0
    for i in range(len(expectedStates)):
        fidelity = qml.math.fidelity_statevector(y_true[i], y_pred[i])
        loss += fidelity
    return np.sqrt(1 - (1/4)*abs(loss))

# Define the correct operations you want the matrix to perform on basis vectors
def target_operations(parameters, inputStates):
    # Reshape the parameters into the matrix form
    parameters = np.reshape(parameters, (num_layers, 5))
    matrix = get_matrix(parameters)

    # Perform matrix multiplication with basis vectors
    results = []
    for i in range(len(inputStates)):
        results.append(np.matmul(matrix, inputStates[i]))

    # Define the target operations you want (modify this based on your specific task)
    target_result = np.array(expectedStates)

    # Calculate the loss as the difference between the obtained result and the target result
    # loss = square_loss(target_result, results)
    loss = f_cnot_loss(target_result, results)
    return loss

# Example: Set the number of basis vectors and their dimensionality
num_vectors = 4
vector_dimension = size_of_vec

# Generate random basis vectors and target result
basis_vectors = np.array(inputStates)
target_result = np.array(expectedStates)

# Flatten the matrix parameters for optimization
initial_parameters = np.ndarray.flatten(get_random_weights(num_layers))

# Use scipy's minimize function to optimize the parameters
result = minimize(target_operations, initial_parameters, args=(basis_vectors,), method='L-BFGS-B')

# Reshape the optimized parameters back into the matrix form
optimized_matrix = get_matrix(result.x.reshape((num_layers, 5)))

print("Optimized Matrix:")
print(optimized_matrix)

predStates = [np.matmul(optimized_matrix, mat) for mat in inputStates]
print(f"f_cnot_loss = {f_cnot_loss(expectedStates, predStates)}")
print(f"square_loss = {square_loss(expectedStates, predStates)}")

Optimized Matrix:
[[-0.27353149-0.96186305j  0.        +0.j          0.        +0.j
  ...  0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j         -0.30504112-0.00295081j  0.29018547+0.27333573j
  ...  0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.36993817-0.06883858j -0.02443   +0.11591914j
  ...  0.        +0.j          0.        +0.j
   0.        +0.j        ]
 ...
 [ 0.        +0.j          0.        +0.j          0.        +0.j
  ... -0.02443   +0.11591914j  0.36993817-0.06883858j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
  ...  0.29018547+0.27333573j -0.30504112-0.00295081j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
  ...  0.        +0.j          0.        +0.j
  -0.27353149-0.96186305j]]
f_cnot_loss = 2.3654860059004343e-07
square_loss = 1.9445606202972602e-27


# Attempted to make a Neural Network model but wasn't working

In [41]:
import tensorflow as tf
import numpy as np

def f_cnot_loss(y_true, y_pred):
    loss = 0
    for i in range(np.size(y_true)//(2**n_qubits)):
        fidelity = qml.math.fidelity_statevector(y_true[i], y_pred[i])
        loss += fidelity
    return np.sqrt(1 - (1/4)*abs(loss))


# Define the correct operations you want the matrix to perform on basis vectors
def target_operations(parameters, inputVectors):
    # Perform matrix multiplication with basis vectors
    # Reshape the parameters into the matrix form
    params = np.reshape(parameters, (num_layers, 5))
    matrix = tf.constant(get_matrix(params), dtype=tf.complex128)
    # results = tf.matmul(matrix, inputVectors, transpose_b=True)
    results = []
    for i in range(np.size(inputVectors)//(2**n_qubits)):
        results.append(np.matmul(matrix, inputVectors[i]))
    results = tf.constant(results, dtype=tf.complex128)

    # Define the target operations you want (modify this based on your specific task)
    target_results = tf.constant(expectedStates, dtype=tf.complex128)

    # Calculate the loss as the mean squared error between the obtained result and the target result
    loss = f_cnot_loss(target_results, results)
    return loss

# Example: Set the number of basis vectors and their dimensionality
num_vectors = 4
vector_dimension = 2**n_qubits

# Generate random basis vectors and target result
inputVectors = tf.constant(inputStates, dtype=tf.complex128)

# Define the matrix as a TensorFlow Variable
parameters = tf.Variable(np.ndarray.flatten(get_random_weights(num_layers)), dtype=tf.float32)

# Use an optimizer to minimize the loss
optimizer = tf.optimizers.Adam(learning_rate=0.01)

# Training loop
epochs = 1000
for epoch in range(epochs):
    with tf.GradientTape() as tape:
        loss = tf.constant(target_operations(parameters, inputVectors))

    gradients = tape.gradient(loss, [parameters])
    optimizer.apply_gradients(zip(gradients, [parameters]))

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.numpy()}")

# Get the optimized matrix
parameters = np.reshape(parameters, (num_layers, 5))
matrix = get_matrix(parameters)
optimized_matrix = matrix.numpy()

print("Optimized Matrix:")
print(optimized_matrix)

ValueError: No gradients provided for any variable: (['Variable:0'],). Provided `grads_and_vars` is ((None, <tf.Variable 'Variable:0' shape=(35,) dtype=float32, numpy=
array([ 2.5269053 ,  0.5212329 ,  0.9739547 ,  2.5969772 ,  0.6503308 ,
        2.1297295 , -1.931953  ,  0.7041187 , -1.3276408 , -2.2411902 ,
        1.2720861 ,  1.5847887 ,  3.0150158 ,  1.2192807 , -1.7325749 ,
       -0.82495993, -1.756478  , -1.1077476 , -2.713036  ,  1.8335059 ,
        1.614816  ,  2.2740006 ,  2.2506728 , -1.1599189 , -2.0895307 ,
        3.0720863 , -0.31033638, -0.30215982, -2.318698  , -3.134845  ,
       -1.3226986 , -2.8566093 ,  1.5320274 ,  0.39243674,  0.13296835],
      dtype=float32)>),).

# Manual matrix multiplication of what the paper suggested

In [7]:
p1 = np.arccos(-1/np.sqrt(3))/np.pi
p2 = np.arcsin(1/3)/np.pi


def Id_n(n):
    assert n >= 0
    if n==0:
        return 1
    temp = I
    for i in range(n-1):
        temp = np.kron(temp, I)
    return temp


bounds = [[3,4],[2,5],[3,4],[2,5],[1,4],[2,5],[1,4],[2,5],[1,4],[2,5],[3,4],[2,5],[3,4]]
operators = [U_ex(p1),
np.kron(U_ex(1/2),U_ex(p2)),
U_ex(1),
np.kron(U_ex(-1/2),U_ex(-1/2)),
np.kron(U_ex(1),U_ex(-1/2)),
np.kron(U_ex(-1/2),U_ex(1)),
np.kron(U_ex(-1/2),U_ex(1/2)),
np.kron(U_ex(-1/2),U_ex(1)),
np.kron(U_ex(1),U_ex(-1/2)),
np.kron(U_ex(-1/2),U_ex(-1/2)),
U_ex(1),
np.kron(U_ex(1/2),U_ex(1-p2)),
U_ex(-p1)]


newOps = []
for i, (start, end) in enumerate(bounds):
    temp = nestedKron(Id_n(start),operators[i], Id_n(5-end))
    newOps.append(temp.copy())

totalOperator = np.eye(2**6)
for op in newOps:
    totalOperator = np.matmul(op,totalOperator)

U_cnot = totalOperator.copy()

def f_cnot_loss(y_true, y_pred):
    loss = 0
    for i in range(np.size(y_true)//(2**n_qubits)):
        fidelity = qml.math.fidelity_statevector(y_true[i], y_pred[i])
        loss += fidelity
    return np.sqrt(1 - (1/4)*abs(loss))


predStates = [np.matmul(U_cnot, mat) for mat in inputStates]
print(f"f_cnot loss = {f_cnot_loss(expectedStates, predStates)}")

f_cnot loss = 0.7189360747413388
